In [1]:
import json, mistune, statistics

In [2]:
with open('data/labelled-issues/closed_issues.json') as f:
  data = json.load(f)

In [3]:
markdown = mistune.create_markdown(renderer=mistune.AstRenderer())

In [4]:
def count_codeblocks(issue, code_type):
  return len(
    list(
      filter(
        lambda node: node['type'] == 'block_code' and (
          (not code_type and not node['info']) or
          (code_type and node['info'] and node['info'].lower() == code_type)
        ),
        markdown(issue['body'])
      )
    )
  )
print('''
  Average number of Scala blocks per issue: {scala_blocks}
  Average number of generic blocks per issue: {generic_blocks}
'''.format(
  scala_blocks = statistics.mean([count_codeblocks(iss, 'scala') for iss in data[:]]),
  generic_blocks = statistics.mean([count_codeblocks(iss, None) for iss in data[:]])
))


  Average number of Scala blocks per issue: 1.143184421534937
  Average number of generic blocks per issue: 0.4361970217640321



In [23]:
def count_codeblocks(raw_body, filter_condition = None):
  if not filter_condition:
    predicate = lambda _: True
  elif isinstance(filter_condition, str):
    if filter_condition == 'empty':
      predicate = lambda node: not node['info']
    else:
      predicate = lambda node: node['info'] and node['info'] == filter_condition
  else:
    predicate = filter_condition
  print(predicate)
  return len([node for node in markdown(raw_body) if predicate(node)])

In [24]:
count_codeblocks(data[3]['body'])

<function count_codeblocks.<locals>.<lambda> at 0x7f94da3e75e0>


14

In [19]:
data[3]['html_url']

'https://github.com/lampepfl/dotty/issues/12830'

In [13]:
markdown(data[3]['body'])

[{'type': 'paragraph',
  'children': [{'type': 'text',
    'text': 'Related: https://github.com/lampepfl/dotty/issues/3969'}]},
 {'type': 'paragraph',
  'children': [{'type': 'text', 'text': 'The '},
   {'type': 'codespan', 'text': 'DottyTypeStealer'},
   {'type': 'text',
    'text': " explained in the document https://dotty.epfl.ch/docs/contributing/workflow.html#inspecting-trees-with-type-stealer doesn't work from "},
   {'type': 'codespan', 'text': 'console'},
   {'type': 'text', 'text': '.'}]},
 {'type': 'heading',
  'children': [{'type': 'text', 'text': 'Compiler version'}],
  'level': 2},
 {'type': 'block_code',
  'text': '$ sbt\nsbt:scala3> scalac -version\n[info] running (fork) dotty.tools.dotc.Main -classpath /Users/tanishiking/Library/Caches/Coursier/v1/https/repo1.maven.org/maven2/org/scala-lang/scala-library/2.13.6/scala-library-2.13.6.jar:/Users/tanishiking/src/github.com/tanishiking/dotty/library/../out/bootstrap/scala3-library-bootstrapped/scala-3.0.2-RC1-bin-SNAPSHOT-no

In [18]:
with open('external.json', 'w') as f:
  json.dump(list(filter(lambda i: i['author_association'] == 'NONE', data)), f, indent=2)